In [63]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, EditedNearestNeighbours, NearMiss

In [64]:
#Importation des données guillaume.txt et création dataframe
data= pd.read_csv('guillaume.txt',sep=';')
data.head()

C:\Users\anicoullau\AppData\Local\Temp\ipykernel_17044\38653645.py:2: DtypeWarning: Columns (1,2,5,6,7,8,9,15,16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv('guillaume.txt',sep=';')


,ZIBZIN,IDAvisAutorisationCheque,FlagImpaye,Montant,DateTransaction,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,...,TauxImpNb_RB,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr1,DiffDateTr2,DiffDateTr3,CA3TRetMtt,CA3TR,Heure
0,A013010004908126703060931,78643044,0,20,2017-02-01 07:32:14,1,0,0,0,551,...,"37,186667890919111","52,076033757361408",0,1,4,4,4,20,0,27134
1,A013011306908024927155000,78643045,0,20,2017-02-01 07:43:37,1,0,0,0,551,...,"48,844716275908937","52,076033757361408",1,2,"1,7976851851851852",4,4,"28,609999999999999","8,6099999999999994",27817
2,A013010002908283134592527,78643046,0,"57,640000000000001",2017-02-01 07:47:38,1,0,0,0,549,...,"73,118279569892479","52,076033757361408",0,1,4,4,4,"57,640000000000001",0,28058
3,A011010002908105209831316,78643047,0,"54,289999999999999",2017-02-01 07:48:48,0,1,1,1,267,...,"110,05692599620494","53,554233554497365",0,1,4,4,4,"54,289999999999999",0,28128
4,A013010041908000125652029,78643048,0,"26,899999999999999",2017-02-01 08:13:27,1,0,0,0,549,...,"45,36831264567185","52,076033757361408",1,1,"1,9971064814814814",4,4,"59,149999999999999","32,25",29607


In [65]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4646774 entries, 0 to 4646773
Data columns (total 23 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   ZIBZIN                    object
 1   IDAvisAutorisationCheque  object
 2   FlagImpaye                object
 3   Montant                   object
 4   DateTransaction           object
 5   CodeDecision              object
 6   VerifianceCPT1            object
 7   VerifianceCPT2            object
 8   VerifianceCPT3            object
 9   D2CB                      object
 10  ScoringFP1                object
 11  ScoringFP2                object
 12  ScoringFP3                object
 13  TauxImpNb_RB              object
 14  TauxImpNB_CPM             object
 15  EcartNumCheq              object
 16  NbrMagasin3J              object
 17  DiffDateTr1               object
 18  DiffDateTr2               object
 19  DiffDateTr3               object
 20  CA3TRetMtt                object
 21  CA3TR   

# Nettoyage des données:

Suppression de la ligne 1956360

In [66]:
data = data.drop(1956360)

Retypage des colonnes

In [67]:
data['ZIBZIN'] = data['ZIBZIN'].astype(str)

In [68]:
data['DateTransaction'] = pd.to_datetime(data['DateTransaction'], format="%Y-%m-%d %H:%M:%S")


In [69]:
data['Year'] = data['DateTransaction'].dt.year
data['Month'] = data['DateTransaction'].dt.month
data['Day'] = data['DateTransaction'].dt.day
data['Hour'] = data['DateTransaction'].dt.hour
data['Minute'] = data['DateTransaction'].dt.minute
data['Second'] = data['DateTransaction'].dt.second

In [70]:
entiers = ['VerifianceCPT1', 'VerifianceCPT2', 'VerifianceCPT3', 'D2CB', 'EcartNumCheq', 'NbrMagasin3J','Heure','CodeDecision']

# Convertir les colonnes en type entier
data[entiers] = data[entiers].astype(int)

In [71]:
reels = ['Montant', 'ScoringFP1', 'ScoringFP2', 'ScoringFP3', 'TauxImpNb_RB', 'TauxImpNB_CPM', 'DiffDateTr1', 'DiffDateTr2', 'DiffDateTr3', 'CA3TRetMtt', 'CA3TR']

data[reels] = data[reels].applymap(lambda x: str(x).replace(',', '.')).astype(float)

C:\Users\anicoullau\AppData\Local\Temp\ipykernel_17044\3100941527.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data[reels] = data[reels].applymap(lambda x: str(x).replace(',', '.')).astype(float)


In [72]:
data['FlagImpaye'] = data['FlagImpaye'].replace({'0': 0, '1': 1})
# Transformez la colonne en facteur (catégorie)
data['FlagImpaye'] = data['FlagImpaye'].astype('category')

In [73]:
data.dtypes

ZIBZIN                              object
IDAvisAutorisationCheque            object
FlagImpaye                        category
Montant                            float64
DateTransaction             datetime64[ns]
CodeDecision                         int32
VerifianceCPT1                       int32
VerifianceCPT2                       int32
VerifianceCPT3                       int32
D2CB                                 int32
ScoringFP1                         float64
ScoringFP2                         float64
ScoringFP3                         float64
TauxImpNb_RB                       float64
TauxImpNB_CPM                      float64
EcartNumCheq                         int32
NbrMagasin3J                         int32
DiffDateTr1                        float64
DiffDateTr2                        float64
DiffDateTr3                        float64
CA3TRetMtt                         float64
CA3TR                              float64
Heure                                int32
Year       

Suppression de colonnes

In [74]:
data = data.drop(columns=['ZIBZIN','IDAvisAutorisationCheque'])

In [75]:
data.head()

,FlagImpaye,Montant,DateTransaction,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,ScoringFP1,ScoringFP2,...,DiffDateTr3,CA3TRetMtt,CA3TR,Heure,Year,Month,Day,Hour,Minute,Second
0,0,20.00,2017-02-01 07:32:14,1,0,0,0,551,0.000000,0.000000,...,4.0,20.00,0.00,27134,2017,2,1,7,32,14
1,0,20.00,2017-02-01 07:43:37,1,0,0,0,551,0.000000,0.000000,...,4.0,28.61,8.61,27817,2017,2,1,7,43,37
2,0,57.64,2017-02-01 07:47:38,1,0,0,0,549,0.000000,0.000000,...,4.0,57.64,0.00,28058,2017,2,1,7,47,38
3,0,54.29,2017-02-01 07:48:48,0,1,1,1,267,0.000000,0.000000,...,4.0,54.29,0.00,28128,2017,2,1,7,48,48
4,0,26.90,2017-02-01 08:13:27,1,0,0,0,549,0.003769,8.586333,...,4.0,59.15,32.25,29607,2017,2,1,8,13,27


Données manquantes: aucunes

In [76]:
print(data.isnull().sum())

FlagImpaye         0
Montant            0
DateTransaction    0
CodeDecision       0
VerifianceCPT1     0
VerifianceCPT2     0
VerifianceCPT3     0
D2CB               0
ScoringFP1         0
ScoringFP2         0
ScoringFP3         0
TauxImpNb_RB       0
TauxImpNB_CPM      0
EcartNumCheq       0
NbrMagasin3J       0
DiffDateTr1        0
DiffDateTr2        0
DiffDateTr3        0
CA3TRetMtt         0
CA3TR              0
Heure              0
Year               0
Month              0
Day                0
Hour               0
Minute             0
Second             0
dtype: int64


In [77]:
print(data.isna().sum())

FlagImpaye         0
Montant            0
DateTransaction    0
CodeDecision       0
VerifianceCPT1     0
VerifianceCPT2     0
VerifianceCPT3     0
D2CB               0
ScoringFP1         0
ScoringFP2         0
ScoringFP3         0
TauxImpNb_RB       0
TauxImpNB_CPM      0
EcartNumCheq       0
NbrMagasin3J       0
DiffDateTr1        0
DiffDateTr2        0
DiffDateTr3        0
CA3TRetMtt         0
CA3TR              0
Heure              0
Year               0
Month              0
Day                0
Hour               0
Minute             0
Second             0
dtype: int64


Distribution des classes

In [78]:
classe= data['FlagImpaye'].value_counts()
print(classe)

FlagImpaye
0    4616778
1      29995
Name: count, dtype: int64


In [79]:
# Calcul du pourcentage de chaque classe
pourcentage_classe_0 = round((classe[0] / len(data)) * 100,2)
pourcentage_classe_1 = round((classe[1] / len(data)) * 100, 2)

print(f'Pourcentage de classe 0 : {pourcentage_classe_0}%')
print(f'Pourcentage de classe 1 : {pourcentage_classe_1}%')


Pourcentage de classe 0 : 99.35%
Pourcentage de classe 1 : 0.65%


# Extraction du fichier nettoyé au format pickle:

In [80]:
import pickle

with open('dataframe.pkl', 'wb') as file:
    pickle.dump(data, file)